In [1]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func, text
from sqlalchemy.orm import sessionmaker

sys.path.append('../..')
sys.path.append('../../../ajna_docs/commons')


In [2]:
from ajna_commons.flask.conf import SQL_URI
from ajna_commons.utils.images import mongo_image
from virasana.db import mongodb as db


2025-09-23 09:50 ajna         INFO     Configuração de log efetuada


2025-09-23 09:50:32,257 ajna         INFO     Configuração de log efetuada


In [3]:
%load_ext autoreload 
%autoreload 2
engine = create_engine(SQL_URI)

# Recuperar informações de REDEX e OPERADORES

In [4]:
# Constantes organizando códigos
LISTA_OPERADORES = "('8931356', '8931359', '8931404', '8931318')"  # Terminais exportadores BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO
LISTA_REDEX = "('8931309', '8933204', '8931404')" # Redex da Localfrio/Movecta, Redex da Santos Brasil Clia e Redex da DPW/EMBRAPORT
FILTRO_LIKE_REDEX = "'89327%'"  # Abrange todos os outros Redex santistas

from datetime import datetime, timedelta

agora = datetime.now()
ha_X_dias = agora - timedelta(days=7)

# Parâmetros de data
data_final = agora.strftime('%Y-%m-%d %H:%M:%S')
data_inicial = ha_X_dias.strftime('%Y-%m-%d %H:%M:%S')

In [5]:
# SQL para entrada_operadores (alias e)
sql_entrada_operadores = f"""
SELECT 
    id, placa, placaSemirreboque, numeroConteiner, codigoRecinto, dataHoraOcorrencia, operacao, direcao, 
    cnpjTransportador, cpfMotorista, nomeMotorista, listaNfe, numeroConhecimento, vazioConteiner
FROM apirecintos_acessosveiculo
WHERE 
    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND operacao = 'C'
    AND direcao = 'E'
    AND codigoRecinto IN {LISTA_OPERADORES}
    AND numeroConteiner !='' 
    AND vazioConteiner = 0 
"""

# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'E' é porque queremos a Entrada no Operador Terminal Exportador (BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO)
# Este 'C' é porque é registro de acesso (entrada) e não agendamento
# Este '0' indica que o conteiner chegou estufado (cheio) ao Operador Terminal Exportador (BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO)

In [6]:
# SQL para pesagem_operador - pesagem com recintos dos operadores (terminais exportadores)
sql_pesagem_operador = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 18000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND codigoRecinto IN {LISTA_OPERADORES}
"""
# Se o caminhao com conteiner pesa 18 toneladas ou mais, então, ele está carregado (não é vazio)

In [7]:
# SQL para pesagem_redex - pesagem com recintos Redex (estufadores)
sql_pesagem_redex = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 18000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND (codigoRecinto IN {LISTA_REDEX} OR codigoRecinto LIKE {FILTRO_LIKE_REDEX})
"""
# Se o caminhao com conteiner pesa 18 toneladas ou mais, então, ele está carregado (não é vazio)

In [8]:
# SQL com JOIN para filtrar saidas do REDEX seguidas de entrada no Terminal Exportador (Operador portuário)
sql_saida_redex = f"""
SELECT DISTINCT
    c.id, c.placa, c.placaSemirreboque, c.numeroConteiner, c.vazioConteiner,
    c.codigoRecinto, c.cpfMotorista, c.nomeMotorista, c.dataHoraOcorrencia, c.direcao
FROM apirecintos_acessosveiculo AS c
JOIN apirecintos_acessosveiculo AS e
    ON e.numeroConteiner = c.numeroConteiner AND e.placaSemirreboque = c.placaSemirreboque
    AND e.dataHoraOcorrencia >= c.dataHoraOcorrencia
    AND e.direcao = 'E'
    AND e.operacao = 'C'
    AND (e.codigoRecinto NOT IN {LISTA_REDEX} AND e.codigoRecinto NOT LIKE {FILTRO_LIKE_REDEX})
WHERE 
    c.dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND c.operacao = 'C'
    AND c.direcao = 'S'
    AND (c.codigoRecinto IN {LISTA_REDEX} OR c.codigoRecinto LIKE {FILTRO_LIKE_REDEX})
    AND c.numeroConteiner != ''    
"""
# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'S' é porque queremos a Saida do Redex estufador
# Este 'C' é porque é registro de acesso (saída) e não agendamento
# A tabela c. é a tabela de ACESSO DE SAÍDA do REDEX no API RECINTOS
# A tabela e. é a tabela de ACESSO DE ENTRADA no TERMINAL EXPORTADOR (Operador Portuário)
# A lógica é que o caminhão com conteiner SAI DO REDEX ANTES DE ENTRAR NO TERMINAL EXPORTADOR (por isso o maior igual)

In [9]:
# Execução das consultas com parâmetros
df_entrada_operadores = pd.read_sql(
    text(sql_entrada_operadores), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [10]:
df_entrada_operadores

,id,placa,placaSemirreboque,numeroConteiner,codigoRecinto,dataHoraOcorrencia,operacao,direcao,cnpjTransportador,cpfMotorista,nomeMotorista,listaNfe,numeroConhecimento,vazioConteiner
0,12423429,KUK3G30,TLH0E79,TLLU7659418,8931318,2025-09-16 09:50:44,C,E,01896217000194,65877713000,DOUGLAS DE SOUZA,,,0
1,12423434,JOL8A32,TKK7G03,MSMU5599080,8931318,2025-09-16 09:55:20,C,E,01896217000194,61281360449,JOSE SEVERINO DE ARRUDA,,,0
2,12423435,CUD4902,FOE4388,HMMU6790052,8931318,2025-09-16 09:56:49,C,E,12014047000193,10430518803,ALDO PASCOAL SOARES JUNIOR,,,0
3,12423441,BBG4E04,ETU4H57,AMCU9375063,8931318,2025-09-16 10:15:03,C,E,21425093000176,31432316885,MARCOS ROBERTO DE SOUZA,,,0
4,12423442,DTV6B73,FEI3E07,MSNU5839435,8931318,2025-09-16 10:18:54,C,E,01896217000194,31174972858,LEANDRO HENRIQUE POMBAL CORREA,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19500,12612816,ECS0E09,CUA6D64,MNBU3040757,8931404,2025-09-22 23:49:21,C,E,07755311000100,27179572861,RONALDO SILVA DE SOUZA,35250913171927000217550010000107861865370694,,0
19501,12612820,TJB7I98,GCM9G91,MRKU4811566,8931404,2025-09-22 23:50:29,C,E,58317751000205,44006556870,MATHEUS DO CARMO LIRA,,,0
19502,12612834,BYF6D98,DFG6584,CAAU9115458,8931404,2025-09-22 23:52:31,C,E,49871560000100,44884456831,FELLIPE CAMPOS SILVA DE OLIVEI,35250906538082000370550060000448121000040907,,0
19503,12612830,PYD9E86,TLO1D69,TLLU7882442,8931404,2025-09-22 23:53:04,C,E,21425093000176,42596590816,JACKSON ANTONIO DE SOUZA LAZAR,,,0


In [11]:
df_saida_redex = pd.read_sql(
    text(sql_saida_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [12]:
df_saida_redex

,id,placa,placaSemirreboque,numeroConteiner,vazioConteiner,codigoRecinto,cpfMotorista,nomeMotorista,dataHoraOcorrencia,direcao
0,12437639,OZD7B52,TLO9D24,CAAU6294241,0,8932796,21864622814,KLEBER CUCO DA SILVA,2025-09-16 09:50:37,S
1,12438148,DVS1I68,CUA5F95,MSDU2103970,1,8932797,37415733886,JEFFERSON DE SOUZA FELIX RODRIGUES,2025-09-16 09:52:24,S
2,12436228,CUA7G34,DPC8847,TCNU8642527,0,8932710,00696425165,LEANDRO DA SILVA BRITZKE,2025-09-16 09:53:30,S
3,12437649,GAB8G78,FWW7B90,MRSU3437562,0,8932796,52695714807,HALISTON CARNEIRO GOMES,2025-09-16 09:57:49,S
4,12437645,LAO2F31,BTR0B28,TCNU4122677,0,8932796,09774969880,SILVIO ETTINGER TORAL,2025-09-16 10:01:52,S
...,...,...,...,...,...,...,...,...,...,...
2084,12618558,RTN5I35,ECM3526,MRKU9555446,1,8932793,27919728894,MARCIO ANDRE BATISTA,2025-09-22 22:58:47,S
2085,12617715,BTO5G02,CUA6I29,SUDU6952928,1,8932724,38649883818,PAULO VITOR,2025-09-22 23:01:05,S
2086,12618559,RTF6C39,ECM3502,SEGU2176359,1,8932793,07018548802,MIGUEL PEREIRA DOS SANTOS,2025-09-22 23:01:58,S
2087,12618560,QRM9G78,EON0605,FXLU2237256,1,8932793,09791324875,CLODOALDO FERREIRA DA SILVA,2025-09-22 23:05:18,S


In [13]:
df_saidas_entradas = pd.merge(df_entrada_operadores, df_saida_redex, how='inner', on=['placaSemirreboque', 'numeroConteiner'] , suffixes=('_E_Operador', '_S_REDEX'))
# Este 'inner' faz com que a tabela final obtida saidas_entradas só tenham dados que tem em Entrada Operador e Saída Rede
# Dados que não tenham ligação correspondente (são nulos) NÃO aparecerão
# Se você quiser que os não correspondentes também apareçam troque INNER por LEFT

In [14]:
df_saidas_entradas['transitTime'] = (df_saidas_entradas['dataHoraOcorrencia_E_Operador'] - 
                                     df_saidas_entradas['dataHoraOcorrencia_S_REDEX']).dt.total_seconds() / 3600


In [15]:
# df_saidas_entradas

In [16]:
#!pip install tqdm
from tqdm.notebook import tqdm

def buscar_id_fs_files(collection, numero_conteiner, data_ocorrencia):
    filtro = {
        'metadata.numeroinformado': numero_conteiner,
        'metadata.dataescaneamento': {
            '$gte': data_ocorrencia - timedelta(hours=3),
            '$lte': data_ocorrencia + timedelta(hours=6)
        }
    }
    resultado = collection.find_one(filtro, {'_id': 1})
    if resultado:
        return resultado['_id']
    return None

# Iterar sobre as linhas do DataFrame e aplicar busca
def mapear_ids_mongo(collection, df):
    ids_fs_files = []
    cont_sucesso = 0
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Buscando ids no MongoDB"):
        numero = row['numeroConteiner']
        data = row['dataHoraOcorrencia_E_Operador']
        id_fs = buscar_id_fs_files(collection, numero, data)
        ids_fs_files.append(id_fs)
        cont_sucesso += 1 if id_fs else 0
    df['fs_files_id'] = ids_fs_files
    print(f'Registros: {len(df)}, imagens: {cont_sucesso}')

In [17]:
mapear_ids_mongo(db['fs.files'], df_saidas_entradas)

Buscando ids no MongoDB:   0%|          | 0/2049 [00:00<?, ?it/s]

Registros: 2049, imagens: 1005


In [18]:
# df_saidas_entradas.transitTime.describe()
df_saidas_entradas.rename(columns={'placa': 'placa', 'placaSemirreboque': 'placa do semireboque', 'numeroConteiner': 'contêiner', 'vazioConteiner' : 'conteinervazio',
                         'codigoRecinto_S_REDEX': 'Redex', 'dataHoraOcorrencia_S_REDEX': 'Saída REDEX','placaSemirreboque': 'placa do semireboque',
                         'codigoRecinto_E_Operador': 'Operador', 'dataHoraOcorrencia_E_Operador': 'Entrada Operador',
                         'transitTime': 'transitTime', 'fs_files_id': '_id'}, inplace=True)
df_saidas_entradas.drop(['operacao', 'direcao_E_Operador', 'listaNfe', 'numeroConhecimento', 'direcao_S_REDEX'], axis=1)
df_saidas_entradas

,id_E_Operador,placa_E_Operador,placa do semireboque,contêiner,Operador,Entrada Operador,operacao,direcao_E_Operador,cnpjTransportador,cpfMotorista_E_Operador,...,id_S_REDEX,placa_S_REDEX,vazioConteiner_S_REDEX,Redex,cpfMotorista_S_REDEX,nomeMotorista_S_REDEX,Saída REDEX,direcao_S_REDEX,transitTime,_id
0,12423469,EWJ9661,FTU5124,KOCU4524804,8931318,2025-09-16 11:19:54,C,E,12014047000193,90643925449,...,12434555,EWJ9661,0,8931404,90643925449,ANDRE TEIXEIRA DA SILVA SANTOS,2025-09-16 10:16:34,S,1.055556,68ca98be4b6fcba5bbb13a1d
1,12423491,FEI3I38,FUY9846,HMMU5519489,8931318,2025-09-16 11:54:34,C,E,12014047000193,17898192885,...,12434571,FEI3I38,0,8931404,17898192885,ANTONIO CARLOS BRAGA,2025-09-16 10:22:29,S,1.534722,68ca98be4b6fcba5bbb13a18
2,12423598,KJQ0F31,FUI4356,MOFU6706340,8931318,2025-09-16 17:01:37,C,E,12014047000193,08923508810,...,12435152,KJQ0F31,0,8931404,08923508810,PEDRO DE SOUZA,2025-09-16 16:02:10,S,0.990833,68ca990e8e8f95cb27bf8bac
3,12423597,CUD4902,FOE4388,KOCU4247422,8931318,2025-09-16 17:01:42,C,E,12014047000193,10430518803,...,12435144,CUD4902,0,8931404,10430518803,ALDO PASCOAL SOARES JUNIOR,2025-09-16 15:59:01,S,1.044722,68ca990e8e8f95cb27bf8ba7
4,12423607,EGK6354,MHT6E19,TCLU2768486,8931318,2025-09-16 17:11:33,C,E,12014047000193,06160686461,...,12435164,EGK6354,0,8931404,06160686461,VALDEMIR FERNANDO DA SILVA,2025-09-16 16:07:06,S,1.074167,68ca990e8e8f95cb27bf8bc1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044,12609206,CUA6H57,BXH0J81,MRKU2031391,8931359,2025-09-22 23:27:15,C,E,57246266000136,24978468809,...,12617713,CUA6H57,1,8932724,24978468809,WILLIAM MARCIO SANTOS MEDEIROS,2025-09-22 22:53:41,S,0.559444,68d284af40adeeb539a10e0e
2045,12609207,BTO5G02,CUA6I29,SUDU6952928,8931359,2025-09-22 23:27:42,C,E,57246266000136,38649883818,...,12617715,BTO5G02,1,8932724,38649883818,PAULO VITOR,2025-09-22 23:01:05,S,0.443611,68d284b140adeeb539a10f0d
2046,12609209,RTF6C39,ECM3502,SEGU2176359,8931359,2025-09-22 23:28:18,C,E,07275520000156,07018548802,...,12618559,RTF6C39,1,8932793,07018548802,MIGUEL PEREIRA DOS SANTOS,2025-09-22 23:01:58,S,0.438889,None
2047,12609218,QRM9G78,EON0605,FXLU2237256,8931359,2025-09-22 23:30:26,C,E,07275520000156,09791324875,...,12618560,QRM9G78,1,8932793,09791324875,CLODOALDO FERREIRA DA SILVA,2025-09-22 23:05:18,S,0.418889,None


In [19]:
from IPython.display import display, HTML

def get_outliers(df):
    mean = df['transitTime'].mean()
    std = df['transitTime'].std()
    outliers = df[(df['transitTime'] < mean - 1 * std) | (df['transitTime'] > mean + 1 * std)]
    return outliers

def generate_html(_id):
    url = f"https://ajna1.rfoc.srf/virasana/file?_id={_id}"
    img = f"https://ajna1.rfoc.srf/virasana/imagens_cmap?_id={_id}"
    return f'<a href="{url}"><img src="{img}" width="200"></a>'
    
outliers = get_outliers(df_saidas_entradas)[['placa do semireboque', 'contêiner', 'Redex', 'Saída REDEX',
                                  'Operador', 'Entrada Operador', 'transitTime', '_id']]

outliers['html'] = outliers['_id'].apply(generate_html)
display(HTML(outliers.to_html(escape=False)))

,placa do semireboque,contêiner,Redex,Saída REDEX,Operador,Entrada Operador,transitTime,_id,html
41,FKS0889,BMOU9755972,8932722,2025-09-17 19:46:52,8931318,2025-09-19 15:45:48,43.982222,68d1354925f6a1fc5d36ae6a,
232,CPJ5901,MRKU5998826,8931404,2025-09-17 05:01:47,8931356,2025-09-17 18:13:11,13.190000,None,
235,CYY5472,MRSU8708353,8932722,2025-09-17 01:09:20,8931356,2025-09-17 19:24:41,18.255833,None,
264,CUL2G26,MRSU7808759,8931404,2025-09-16 21:08:49,8931356,2025-09-17 22:47:32,25.645278,None,
290,FIX9153,MRSU7277230,8931404,2025-09-16 11:57:25,8931356,2025-09-18 03:02:21,39.082222,None,
299,DCF5B41,MRSU5009537,8931404,2025-09-16 14:08:06,8931356,2025-09-18 03:58:54,37.846667,None,
303,TKE5F38,HAMU1704930,8932722,2025-09-17 04:35:18,8931356,2025-09-18 05:23:37,24.805278,None,
360,BRQ6G50,HAMU1863211,8931404,2025-09-16 18:29:09,8931356,2025-09-18 11:47:13,41.301111,None,
373,GZV4C99,TGBU9888670,8931404,2025-09-17 02:38:49,8931356,2025-09-18 14:13:56,35.585278,None,
375,MLW2F35,HLBU1207960,8931404,2025-09-16 23:26:51,8931356,2025-09-18 14:23:02,38.936389,None,


In [20]:
# Adicionar células clicáveis que levem ao Ajna
df_saidas_entradas['contêiner'] = df_saidas_entradas['contêiner'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/bhadrasana2/consulta_container?numerolote={x}", "{x}")'
)
df_saidas_entradas['_id'] = df_saidas_entradas['_id'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/virasana/file?_id={x}", "{x}")'
)

# df_saidas_entradas

In [21]:
df_saidas_entradas.to_excel('saidas_entradas.xlsx', index=False)

In [22]:
df_pesagem_operador = pd.read_sql(
    text(sql_pesagem_operador), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

df_pesagem_redex = pd.read_sql(
    text(sql_pesagem_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)



# Seguem os merges e filtros conforme a lógica desejada especificada anteriormente
# Por exemplo, aqui unimos pesagem_redex com saida_redex e depois com entrada_operadores conforme passo a passo
df_pesagem_saida = pd.merge(df_pesagem_redex, df_saida_redex, on=['placaSemirreboque', 'codigoRecinto', 'numeroConteiner'], suffixes=('_pesagem', '_saida'))

df_pesagem_saida = df_pesagem_saida[
    (df_pesagem_saida['dataHoraOcorrencia_pesagem'] <= df_pesagem_saida['dataHoraOcorrencia_saida']) &
    ((df_pesagem_saida['dataHoraOcorrencia_saida'] - df_pesagem_saida['dataHoraOcorrencia_pesagem']).dt.days <= 7) &
    (df_pesagem_saida['direcao'] == 'S')
]

df_final = pd.merge(df_pesagem_saida, df_entrada_operadores, left_on=['placaSemirreboque', 'numeroConteiner'], right_on=['placaSemirreboque', 'numeroConteiner'], suffixes=('', '_entrada'))

df_final = df_final[
    ((df_final['dataHoraOcorrencia'] - df_final['dataHoraOcorrencia_saida']).dt.days <= 7) &
    (df_final['codigoRecinto'] != df_final['codigoRecinto_entrada']) 
]

df_final = df_final.drop_duplicates()

# Seleção de colunas para output
output_cols = [
    'dataHoraOcorrencia_pesagem', 'id_pesagem', 'placa', 'placaSemirreboque', 'pesoBrutoBalanca', 'numeroConteiner', 'codigoRecinto_pesagem',
    'dataHoraOcorrencia_saida', 'placa', 'codigoRecinto_saida', 'direcao', 'numeroConteiner',
    'dataHoraOcorrencia', 'placa', 'placaSemirreboque', 'id', 'cnpjTransportador', 'cpfMotorista', 'nomeMotorista', 'codigoRecinto', 'operacao', 'direcao', 'numeroConteiner', 'listaNfe', 'numeroConhecimento'
]

df_result = df_final[output_cols]

print(df_result.head())

KeyError: "['codigoRecinto_pesagem', 'codigoRecinto_saida'] not in index"